Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE: MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework: MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

MindSPONGE tutorial protein 02: Add solvent and Periodic boundary conditions.

In [1]:
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'
from mindspore import context, nn
import sys
sys.path.append('../../../src')
from sponge import ForceField
from sponge.system import get_molecule
from sponge import set_global_units, WithEnergyCell
from sponge.core import Sponge
from sponge.callback import WriteH5MD, RunInfo

[WARNING] ME(21386:140484086830272,MainProcess):2023-08-15-16:33:37.606.970 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21386:140484086830272,MainProcess):2023-08-15-16:33:37.608.118 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21386:140484086830272,MainProcess):2023-08-15-16:33:37.608.559 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21386:140484086830272,MainProcess):2023-08-15-16:33:37.608.952 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21386:140484086830272,MainProcess):2023-08-15-16:33:37.609.328 [mindspore/common/api.py:897] 'mindspore.ms_function

In [2]:
context.set_context(mode=context.GRAPH_MODE)
set_global_units('nm', 'kj/mol')

load pdb file and add solvent

In [3]:
pdb_name = './case1.pdb'
out_pdb = './case1_sol.pdb'
mol = get_molecule(pdb_name, template=['protein0.yaml'], rebuild_hydrogen=True)
mol.fill_water(edge=0.4, pdb_out=out_pdb, template='water.spce.yaml')

[MindSPONGE] Adding 57 hydrogen atoms for the protein molecule in 0.003 seconds.
[MindSPONGE] The box size used when filling water is: [2.1184928 2.285693  1.549593 ]
[MindSPONGE] The edge gap along x axis is 0.4.
[MindSPONGE] The edge gap along y axis is 0.3999999523162842.
[MindSPONGE] The edge gap along z axis is 0.4.
[MindSPONGE] Totally 172 waters is added to the system!


/tmp/ipykernel_21386/3903672113.py:4: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  mol.fill_water(edge=0.4, pdb_out=out_pdb, template='water.spce.yaml')


[MindSPONGE] Adding water molecule task finished!


_MoleculeFromPDB<>

protein use amber.FF14SB forcefield, while solvent use SPCE forcefield

In [4]:
energy = ForceField(mol, parameters=['AMBER.FF14SB', 'SPCE'], use_pme=False)

do the minimization

In [5]:
min_opt = nn.Adam(mol.trainable_params(), 1e-3)
sim = WithEnergyCell(mol, energy)
md = Sponge(sim, optimizer=min_opt)
run_info = RunInfo(10)
cb_h5md = WriteH5MD(mol, './p02.h5md', save_freq=10, save_last_pdb='./p02.pdb', write_image=False)
md.run(800, callbacks=[run_info, cb_h5md])

print('The final pbc box size is: {}'.format(mol.pbc_box.asnumpy()))

[MindSPONGE] Started simulation at 2023-08-15 16:33:46
[MindSPONGE] Step: 0, E_pot: 24502.51
[MindSPONGE] Step: 10, E_pot: 18154.873
[MindSPONGE] Step: 20, E_pot: 16701.99
[MindSPONGE] Step: 30, E_pot: 15318.471
[MindSPONGE] Step: 40, E_pot: 13748.047
[MindSPONGE] Step: 50, E_pot: 12375.297
[MindSPONGE] Step: 60, E_pot: 11392.975
[MindSPONGE] Step: 70, E_pot: 10696.414
[MindSPONGE] Step: 80, E_pot: 10149.101
[MindSPONGE] Step: 90, E_pot: 9693.352
[MindSPONGE] Step: 100, E_pot: 9311.222
[MindSPONGE] Step: 110, E_pot: 8999.063
[MindSPONGE] Step: 120, E_pot: 8740.288
[MindSPONGE] Step: 130, E_pot: 8517.707
[MindSPONGE] Step: 140, E_pot: 8332.541
[MindSPONGE] Step: 150, E_pot: 8181.0737
[MindSPONGE] Step: 160, E_pot: 8049.1924
[MindSPONGE] Step: 170, E_pot: 7928.546
[MindSPONGE] Step: 180, E_pot: 7815.838
[MindSPONGE] Step: 190, E_pot: 7715.084
[MindSPONGE] Step: 200, E_pot: 7625.1533
[MindSPONGE] Step: 210, E_pot: 7540.8813
[MindSPONGE] Step: 220, E_pot: 7464.2354
[MindSPONGE] Step: 230, 